In [1]:
import sys
#sys.path.insert(1, '/home/linus/projects/fdl/crisp')

In [2]:
import os
import pandas as pd
if os.getcwd() != '/home/jupyter/crisp':
    os.chdir("..")
%pwd

'/home/jupyter/crisp'

In [3]:
from models.SimpleNonLinearInvariantRiskMinimization import SimpleNonLinearInvariantRiskMinimization
from models.LinearInvariantRiskMinimization import LinearInvariantRiskMinimization
from dataio.DataFrameDataset import *
from dataio.datasets import *

In [4]:
from synthetic.facebook_synthetic_data_generator import generator_example

dim_inv = 6
dim_spu = 60
n_exp_train = 1000 #int(2e2)
n_exp_test = 300
n_env = 5
save_dir = 'data/synthetic'

In [5]:
config = {
    "name": "Example Experiment for AH casual ensemble",
    "short_name": "ah_experiment_notebook",
    "bucket_project": "fdl-us-astronaut-health",
    "bucket_name": "ah_21_data",
    "bucket_path": "gs://ah_21_data",
    "verbose": 1,
    "test_val_split": [0.1, 0.1],
    "per_variant_experiment": False,
    "data_options": {
        #'dataset_fp': '../data/test_multiclass.pkl',
        'dataset_fp' : None,
        'output_data_regime' : ["real-valued", "binary", "binary", "real-valued", "multi-class"],
        'subject_keys': 'Subj_ID',
        'targets': ['Target'],
 #        'predictors': ['All'],
        'predictors': None,
        'environments': ['env_split'],
        'exclude': ['Subj_ID'],
        'synthetic_train_test_split' : True,
    },
    "feature_selection_options": {
        "max_features": 20,
        "verbose": 0,
        "seed": 12
    },
    "ensemble_options": {
        "models": ["ERM", "RF", "ICP", "IRM", "DCF", "ITE", "LIRM", "NLICP"]
    },
    "use_cloud":False,
    "results_directory": "results/"
}
data_config = config['data_options']

In [6]:
LIRM_options = config.get("ensemble_options").get('LIRM', {})
LIRM_args = {
            "use_icp_initialization": False,
            "verbose": 0,
            "n_iterations": 5000, # 1000
            "seed": 0,
            "lr": 0.005, # 0.001
            "cuda": False,
    "output_dim" : 1
        }
LIRM_args.update(LIRM_options)

In [7]:
def get_dataset_for_synthetic_experiment(n_example, dim_inv, dim_spu, n_exp_train, n_exp_test, n_env, save_dir):
    # training data
    generator_example(n_example, dim_inv, dim_spu, n_exp_train, n_env ,save_dir, False)
    outfile_train = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_dim_unc_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, 0, n_exp_train, n_env, False))

    # test data
    generator_example(n_example, dim_inv, dim_spu, n_exp_test, n_env ,save_dir, True)
    outfile_test = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_dim_unc_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, 0, n_exp_test, n_env, True))
    
    return outfile_train, outfile_test

In [8]:
results = {}

for n_example in [1]:#,2,3,4,5]:
    output_data_regime = config["data_options"]["output_data_regime"][n_example-1]
    print("\n\nn_example:", n_example, "output_data_regime:", output_data_regime, "\n\n")
    
    outfile_train, outfile_test = get_dataset_for_synthetic_experiment(n_example, dim_inv, dim_spu, n_exp_train, n_exp_test, n_env, save_dir)
    df_train = pd.read_pickle(outfile_train)
    df_test = pd.read_pickle(outfile_test)

    config["data_options"]["dataset_fp_train"] = outfile_train
    config["data_options"]["dataset_fp_test"] = outfile_test
    config["data_options"]["predictors"] = list(df_train.columns[0:dim_inv+dim_spu])



n_example: 1 output_data_regime: real-valued 


Environments variables: {'E0': 0.1, 'E1': 1.5, 'E2': 2, 'E3': 0.30815539086172433, 'E4': 2.016811860476012}
Generated Synthetic Data according to the Facebook setup Example: 1
     df with  66  columns
Environments variables: {'E0': 0.1, 'E1': 1.5, 'E2': 2, 'E3': 0.30815539086172433, 'E4': 2.016811860476012}
Generated Synthetic Data according to the Facebook setup Example: 1
     df with  66  columns


In [9]:
environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

LIRM_args["target"] = config["data_options"]["targets"]
LIRM_args["output_data_regime"] = output_data_regime
LIRM_args["columns"] = test_dataset.predictor_columns

Running a per sample experiment
Using synthetic dataset train/test split
Loaded  5  train environments
Env  0  has  200  samples
X shape  (200, 66)  y shape  (200, 1)
Env  1  has  200  samples
X shape  (200, 66)  y shape  (200, 1)
Env  2  has  200  samples
X shape  (200, 66)  y shape  (200, 1)
Env  3  has  200  samples
X shape  (200, 66)  y shape  (200, 1)
Env  4  has  200  samples
X shape  (200, 66)  y shape  (200, 1)
Loaded test set, X shape: (300, 66)  y shape:  (300, 1)


In [10]:
#lirm = SimpleNonLinearInvariantRiskMinimization(environment_datasets, val_dataset, test_dataset, LIRM_args)
lirm = SimpleNonLinearInvariantRiskMinimization(environment_datasets, test_dataset, test_dataset, LIRM_args)
#lirm = LinearInvariantRiskMinimization(environment_datasets, test_dataset, test_dataset, LIRM_args)

lirm_results_dict = lirm.results()
results[n_example] = lirm_results_dict

Start simple nonlinear-IRM training procedure


In [11]:
lirm_results_dict.keys()

dict_keys(['test_logits', 'test_acc', 'test_nll', 'test_probs', 'test_labels', 'feature_coeffients', 'loss_over_time', 'acc_over_time', 'to_bucket'])

In [12]:
print(results.get(n_example).keys())
"acc/r2:", results.get(n_example).get("test_acc")

dict_keys(['test_logits', 'test_acc', 'test_nll', 'test_probs', 'test_labels', 'feature_coeffients', 'loss_over_time', 'acc_over_time', 'to_bucket'])


('acc/r2:', 12.107722)

In [13]:
result = results.get(n_example).get("to_bucket")

if output_data_regime == "multi-class":
    r = sorted(zip(result.get("features"), np.abs(result.get("coefficients")).sum(axis=1)), key=lambda x: x[1], reverse=True)
else:
    r = sorted(zip(result.get("features"), result.get("coefficients")), key=lambda x: abs(x[1]), reverse=True)
r[:10]

[('Non_causal_58', 0.02112496830523014),
 ('Non_causal_46', 0.017824701964855194),
 ('Non_causal_3', 0.01753070019185543),
 ('Non_causal_24', 0.017273899167776108),
 ('Non_causal_8', 0.016998829320073128),
 ('Non_causal_42', 0.016971666365861893),
 ('Causal_4', 0.016815656796097755),
 ('Causal_5', 0.016711652278900146),
 ('Non_causal_17', 0.016658736392855644),
 ('Non_causal_53', 0.016522562131285667)]

In [14]:
grads = lirm.get_input_gradients()

input_gradients: torch.Size([1000, 66])


In [15]:
features = result.get("features")

In [16]:
sorted(zip(features, grads), key=lambda x: abs(x[1]), reverse=True)[:10]

[('Causal_4', tensor(0.0110)),
 ('Non_causal_24', tensor(0.0109)),
 ('Non_causal_48', tensor(0.0104)),
 ('Non_causal_57', tensor(0.0098)),
 ('Non_causal_13', tensor(0.0097)),
 ('Non_causal_58', tensor(0.0097)),
 ('Non_causal_18', tensor(0.0096)),
 ('Non_causal_2', tensor(0.0096)),
 ('Non_causal_15', tensor(0.0096)),
 ('Non_causal_8', tensor(0.0095))]